In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os


import matplotlib.pyplot as plt
import seaborn as sns


DATA_PATH = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
SEED = 42


def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
train = pd.read_csv(f"{DATA_PATH}train.csv")
test = pd.read_csv(f"{DATA_PATH}test.csv")
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

In [ ]:
#### 노트북 셀 가로 넓게 보기
from IPython.core.display import display, HTML

notebook_config = ""
notebook_config += "<style>"
notebook_config += "  .container { width:90% !important; }"
notebook_config += "</style>"
HTML(notebook_config)

In [ ]:
# SEED 및 프로그램 버전 설정
IS_COLAB = True
PROGRAM_VERSION = "13"

In [ ]:
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install bayesian-optimization


In [ ]:
!pip install scikit-optimize


In [ ]:
!pip install CatBoost

In [ ]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings("ignore")
import shutil
import time
from scipy.stats import gmean
from itertools import combinations
from tqdm import tqdm
tqdm.pandas()

import tensorflow as tf
from tensorflow import keras


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import TimeDistributed, LayerNormalization
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

# Modeling
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import PassiveAggressiveRegressor


from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor

# # AdaBoost
from sklearn.ensemble import AdaBoostRegressor

# XGBoost
from xgboost import plot_importance
from xgboost import XGBRegressor

# lightgbm
from lightgbm import LGBMRegressor

# # catboost
from catboost import CatBoostRegressor

# metric
from sklearn.metrics import mean_absolute_error

from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV


import keras

print(tf.__version__)
print(keras.__version__)


2.13.0
2.13.1


In [ ]:
!pip install rdkit

In [ ]:
!pip install mordred


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.ipython_useSVG = True
# IPythonConsole.molSize = 500, 500

In [ ]:
# 일단 colab 에서 실행하는 것으로 설정
DATA_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
SUBMISSION_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
CLEANED_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'
MODEL_DIR = '/content/drive/MyDrive/데이콘 캐글 컴페티션/2023신약개발/data/'

In [ ]:
train_original = pd.read_csv(f'{DATA_DIR}train.csv')
# dev = pd.read_csv(f'{DATA_DIR}dev.csv')
test = pd.read_csv(f'{DATA_DIR}test.csv')
submission = pd.read_csv(f'{DATA_DIR}sample_submission.csv')

train = train_original # pd.concat([train_original, dev], axis=0)

train_original.shape, train.shape, test.shape

((3498, 11), (3498, 11), (483, 9))

In [ ]:
calc = Calculator(descriptors, ignore_3D=False)
len(calc.descriptors)

1826

In [ ]:
# x_train = calc.pandas([Chem.MolFromSmiles(x) for x in train.SMILES])
# x_train

In [ ]:
# train_complete = pd.concat([train, x_train], axis=1)
# train_complete

In [ ]:
# x_test = calc.pandas([Chem.MolFromSmiles(x) for x in test.SMILES])
# x_test

In [ ]:
# test_complete = pd.concat([test, x_test], axis=1)
# test_complete

In [ ]:
# train_complete.to_csv(f"{CLEANED_DIR}train_mordred_1835.csv", index=False)
# test_complete.to_csv(f"{CLEANED_DIR}test_mordred_1835.csv", index=False)

In [ ]:
train = pd.read_csv(f'{CLEANED_DIR}train_mordred_1835.csv')
test = pd.read_csv(f'{CLEANED_DIR}test_mordred_1835.csv')


### -> 이상치 제거 (총 18개)

In [ ]:
# 1. 극단치만 판단해서 18개 제거
outliers = train[(train['MLM'] > 100.0) | (train['HLM'] > 100.0) | (train['AlogP'] < -3) | (train['Molecular_Weight'] > 800)
     | (train['Num_H_Acceptors'] > 14) | (train['Num_H_Donors'] > 9) | (train['Num_RotatableBonds'] > 20)
     | (train['LogD'] < -4) | (train['Molecular_PolarSurfaceArea'] > 250)]


In [ ]:
outliers.index

Int64Index([ 179,  662,  834,  983, 1092, 1172, 1239, 1584, 2159, 2258, 2367,
            2410, 2586, 2711, 2948, 3157, 3247, 3403],
           dtype='int64')

In [ ]:
train = train.drop(index=outliers.index, axis=0)

In [ ]:
train.shape # 3498 -> 3480

(3480, 1837)

In [ ]:
duplicates = train[train.duplicated(subset=['SMILES'], keep=False)]

duplicates.sort_values(by='SMILES')


,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
2276,TRAIN_2276,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,1.535,31.453,3.556,262.309,3,0,4,3.556,...,9.530611,66.421505,262.121846,7.709466,969,23,100.0,112.0,4.444444,4.500000
451,TRAIN_0451,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.310,24.670,3.556,262.309,3,0,4,3.556,...,9.530611,66.421505,262.121846,7.709466,969,23,100.0,112.0,4.444444,4.500000
2891,TRAIN_2891,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,55.950,69.950,2.172,337.372,4,2,3,2.169,...,10.107040,59.750412,337.142641,7.662333,1584,39,130.0,151.0,8.750000,5.444444
543,TRAIN_0543,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,68.485,85.872,2.172,337.372,4,2,3,2.169,...,10.107040,59.750412,337.142641,7.662333,1584,39,130.0,151.0,8.750000,5.444444
837,TRAIN_0837,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,63.522,62.488,2.293,367.428,5,2,3,2.307,...,10.293467,77.575687,367.121529,8.537710,1673,42,144.0,173.0,8.722222,5.500000
366,TRAIN_0366,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,73.740,66.850,2.293,367.428,5,2,3,2.307,...,10.293467,77.575687,367.121529,8.537710,1673,42,144.0,173.0,8.722222,5.500000
1085,TRAIN_1085,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,23.630,71.470,1.684,381.450,7,1,4,1.684,...,10.273222,75.316682,381.147075,7.778512,1839,40,142.0,165.0,10.402778,5.319444
2848,TRAIN_2848,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,40.657,99.900,1.684,381.450,7,1,4,1.684,...,10.273222,75.316682,381.147075,7.778512,1839,40,142.0,165.0,10.402778,5.319444
2096,TRAIN_2096,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,1.880,61.280,2.843,360.490,3,0,4,2.843,...,10.154519,68.711453,360.241293,6.211057,1561,45,134.0,157.0,9.000000,5.750000
1666,TRAIN_1666,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,5.494,25.397,2.843,360.490,3,0,4,2.843,...,10.154519,68.711453,360.241293,6.211057,1561,45,134.0,157.0,9.000000,5.750000


In [ ]:
processed_duplicate = duplicates.groupby('SMILES')['MLM','HLM'].mean().reset_index()
processed_duplicate

,SMILES,MLM,HLM
0,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.9225,28.0615
1,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,62.2175,77.9110
2,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,68.6310,64.6690
3,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,32.1435,85.6850
4,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,3.6870,43.3385
5,CC(C)NC(=O)c1c(Cl)nn(C)c1NC(=O)c1cc(Br)nn1-c1n...,62.1085,68.1015
6,CC1CC(=O)N(c2ccc(-c3cccc(C#N)c3)cc2)N=C1c1ccc(...,43.1700,31.1300
7,CCCCC/N=c1\n(C)c(=O)nc2sccn12,2.3395,36.8145
8,CCOC(=O)CC1(NC(=O)N2Cc3c(sc4c3CCCC4)-n3cccc3C2...,3.4420,3.6015
9,CCc1nc2cc(Br)c(C(=O)OC)nc2n1CC(=O)c1ccccc1,73.5450,1.1345


In [ ]:
tmp = pd.merge(processed_duplicate,duplicates,how='left',on='SMILES')
tmp

,SMILES,MLM_x,HLM_x,id,MLM_y,HLM_y,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.9225,28.0615,TRAIN_0451,0.310,24.670,3.556,262.309,3,0,...,9.530611,66.421505,262.121846,7.709466,969,23,100.0,112.0,4.444444,4.500000
1,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.9225,28.0615,TRAIN_2276,1.535,31.453,3.556,262.309,3,0,...,9.530611,66.421505,262.121846,7.709466,969,23,100.0,112.0,4.444444,4.500000
2,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,62.2175,77.9110,TRAIN_0543,68.485,85.872,2.172,337.372,4,2,...,10.107040,59.750412,337.142641,7.662333,1584,39,130.0,151.0,8.750000,5.444444
3,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,62.2175,77.9110,TRAIN_2891,55.950,69.950,2.172,337.372,4,2,...,10.107040,59.750412,337.142641,7.662333,1584,39,130.0,151.0,8.750000,5.444444
4,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,68.6310,64.6690,TRAIN_0366,73.740,66.850,2.293,367.428,5,2,...,10.293467,77.575687,367.121529,8.537710,1673,42,144.0,173.0,8.722222,5.500000
5,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,68.6310,64.6690,TRAIN_0837,63.522,62.488,2.293,367.428,5,2,...,10.293467,77.575687,367.121529,8.537710,1673,42,144.0,173.0,8.722222,5.500000
6,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,32.1435,85.6850,TRAIN_1085,23.630,71.470,1.684,381.450,7,1,...,10.273222,75.316682,381.147075,7.778512,1839,40,142.0,165.0,10.402778,5.319444
7,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,32.1435,85.6850,TRAIN_2848,40.657,99.900,1.684,381.450,7,1,...,10.273222,75.316682,381.147075,7.778512,1839,40,142.0,165.0,10.402778,5.319444
8,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,3.6870,43.3385,TRAIN_1666,5.494,25.397,2.843,360.490,3,0,...,10.154519,68.711453,360.241293,6.211057,1561,45,134.0,157.0,9.000000,5.750000
9,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,3.6870,43.3385,TRAIN_2096,1.880,61.280,2.843,360.490,3,0,...,10.154519,68.711453,360.241293,6.211057,1561,45,134.0,157.0,9.000000,5.750000


### -> 중복치 제거 (총 26개)

In [ ]:
tmp = tmp.iloc[0::2]  # 첫 번째 행부터 시작하여 2씩 증가하여 행 선택
tmp.drop(['MLM_y','HLM_y'],axis =1,inplace=True)

In [ ]:
# tmp.columns = ['SMILES', 'MLM', 'HLM','id',  'AlogP', 'Molecular_Weight',
#        'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
#        'Molecular_PolarSurfaceArea']

In [ ]:
# processed_duplicate = tmp[['id', 'SMILES', 'MLM', 'HLM', 'AlogP', 'Molecular_Weight',
#        'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
#        'Molecular_PolarSurfaceArea']]

In [ ]:
train = train.drop_duplicates(subset=['SMILES'], keep=False) #기존 train에서 중복치들을 우선 모두 제거
train.shape

(3428, 1837)

In [ ]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,...,10.208580,77.363487,395.052750,10.129558,1615,38,136.0,162.0,10.062500,5.250000
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,...,10.291162,75.955433,359.138225,8.162232,1765,45,144.0,173.0,7.750000,6.027778
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,...,9.677277,66.189153,261.147727,6.872309,795,26,94.0,108.0,6.916667,4.333333
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,...,9.604475,65.335399,284.056385,8.876762,812,25,94.0,107.0,6.916667,4.250000


In [ ]:
tmp.shape

(26, 1837)

In [ ]:
tmp.rename(columns={'HLM_x': 'HLM', 'MLM_x': 'MLM'}, inplace=True)

In [ ]:
train = pd.concat([train, tmp], axis=0, ignore_index=True)
train.shape

(3454, 1837)

In [ ]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.0100,50.6800,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.2700,50.5900,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.5860,80.8920,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.7100,2.0000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.2700,99.9900,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,TRAIN_1833,Cc1ccc(-c2cc(-c3nc4c5ccccc5ncn4n3)[nH]n2)c(C)c1,28.4630,24.1955,4.609,340.381,4,1,2,4.736,...,10.344674,77.430819,340.143645,8.098658,1730,43,148.0,180.0,6.611111,5.500000
3450,TRAIN_0898,Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4...,56.0435,66.1260,4.282,553.480,5,0,7,4.009,...,10.593756,88.164476,552.169496,8.120140,4874,63,204.0,242.0,12.055556,8.222222
3451,TRAIN_2575,Cc1ccccc1-c1nc2ccccc2cc1C(C)n1c(=O)[nH]c2c(S(C...,56.3130,63.7320,4.304,459.520,6,1,4,4.304,...,10.677177,84.226168,459.136511,8.502528,2888,61,186.0,227.0,10.895833,6.944444
3452,TRAIN_0216,N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1,70.2150,93.8400,-1.133,385.417,7,0,4,-1.133,...,10.177818,79.094663,385.175004,7.552451,2262,42,148.0,175.0,8.000000,6.222222


# Mordred Columns확인

## Object Columns (총 575개)
- train, test 하나라도

In [ ]:
train_prop = train.iloc[:, 4:]
test_prop = test.iloc[:, 2:]

target_1 = train['MLM']
target_2 = train['HLM']

In [ ]:
train.shape, test.shape

((3454, 1837), (483, 1835))

In [ ]:
test.columns

Index(['id', 'SMILES', 'AlogP', 'Molecular_Weight', 'Num_H_Acceptors',
       'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       'Molecular_PolarSurfaceArea', 'ABC',
       ...
       'SRW10', 'TSRW10', 'MW', 'AMW', 'WPath', 'WPol', 'Zagreb1', 'Zagreb2',
       'mZagreb1', 'mZagreb2'],
      dtype='object', length=1835)

In [ ]:
train.columns

Index(['id', 'SMILES', 'MLM', 'HLM', 'AlogP', 'Molecular_Weight',
       'Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds', 'LogD',
       ...
       'SRW10', 'TSRW10', 'MW', 'AMW', 'WPath', 'WPol', 'Zagreb1', 'Zagreb2',
       'mZagreb1', 'mZagreb2'],
      dtype='object', length=1837)

In [ ]:
object_columns_train = []

for column in train_prop.columns:
    dtype = train_prop[column].dtype
    print(f"Column: {column}, Data Type: {dtype}")
    if dtype == object :
        object_columns_train.append(column)


Column: AlogP, Data Type: float64
Column: Molecular_Weight, Data Type: float64
Column: Num_H_Acceptors, Data Type: int64
Column: Num_H_Donors, Data Type: int64
Column: Num_RotatableBonds, Data Type: int64
Column: LogD, Data Type: float64
Column: Molecular_PolarSurfaceArea, Data Type: float64
Column: ABC, Data Type: float64
Column: ABCGG, Data Type: float64
Column: nAcid, Data Type: int64
Column: nBase, Data Type: int64
Column: SpAbs_A, Data Type: float64
Column: SpMax_A, Data Type: float64
Column: SpDiam_A, Data Type: float64
Column: SpAD_A, Data Type: float64
Column: SpMAD_A, Data Type: float64
Column: LogEE_A, Data Type: float64
Column: VE1_A, Data Type: float64
Column: VE2_A, Data Type: float64
Column: VE3_A, Data Type: float64
Column: VR1_A, Data Type: float64
Column: VR2_A, Data Type: float64
Column: VR3_A, Data Type: float64
Column: nAromAtom, Data Type: int64
Column: nAromBond, Data Type: int64
Column: nAtom, Data Type: int64
Column: nHeavyAtom, Data Type: int64
Column: nSpiro, 

In [ ]:
object_columns_test = []

for column in test_prop.columns:
    dtype = test_prop[column].dtype
    print(f"Column: {column}, Data Type: {dtype}")
    if dtype == object :
        object_columns_test.append(column)


Column: AlogP, Data Type: float64
Column: Molecular_Weight, Data Type: float64
Column: Num_H_Acceptors, Data Type: int64
Column: Num_H_Donors, Data Type: int64
Column: Num_RotatableBonds, Data Type: int64
Column: LogD, Data Type: float64
Column: Molecular_PolarSurfaceArea, Data Type: float64
Column: ABC, Data Type: float64
Column: ABCGG, Data Type: float64
Column: nAcid, Data Type: int64
Column: nBase, Data Type: int64
Column: SpAbs_A, Data Type: float64
Column: SpMax_A, Data Type: float64
Column: SpDiam_A, Data Type: float64
Column: SpAD_A, Data Type: float64
Column: SpMAD_A, Data Type: float64
Column: LogEE_A, Data Type: float64
Column: VE1_A, Data Type: float64
Column: VE2_A, Data Type: float64
Column: VE3_A, Data Type: float64
Column: VR1_A, Data Type: float64
Column: VR2_A, Data Type: float64
Column: VR3_A, Data Type: float64
Column: nAromAtom, Data Type: int64
Column: nAromBond, Data Type: int64
Column: nAtom, Data Type: int64
Column: nHeavyAtom, Data Type: int64
Column: nSpiro, 

In [ ]:
object_columns = object_columns_train + object_columns_test

object_columns = set(object_columns)
object_columns = list(object_columns)

len(object_columns_train), len(object_columns_test), len(object_columns)

(575, 403, 575)

In [ ]:
row_info = train_prop.iloc[0]
row_info

AlogP                    3.259
Molecular_Weight       400.495
Num_H_Acceptors              5
Num_H_Donors                 2
Num_RotatableBonds           8
                        ...   
WPol                        40
Zagreb1                  142.0
Zagreb2                  165.0
mZagreb1                   9.5
mZagreb2              6.361111
Name: 0, Length: 1833, dtype: object

### 1) pure object columns(str) -> 삭제 (319개)
- object칼럼 중 합계 0 인 칼럼
- 행별로 동일해서 의미없는 object column으로 판단함

In [ ]:
# # 열(column_name)의 숫자 데이터만 추출하여 합계 계산
# numeric_values = pd.to_numeric(train_prop[object_columns[0]], errors='coerce')
# column_sum = numeric_values.sum()
# print(i, column_sum)

In [ ]:
pure_object_cols_train = []

for i in range(len(object_columns)):
    numeric_values = pd.to_numeric(train_prop[object_columns[i]], errors='coerce')
    column_sum = numeric_values.sum()
    print('column',i,':', column_sum)
    if column_sum == 0.0:
        pure_object_cols_train.append(object_columns[i])


column 0 : 0.0
column 1 : 0.0
column 2 : -885.5201355572697
column 3 : 0.0
column 4 : 472.83690430857007
column 5 : 0.0
column 6 : -170.25523047977708
column 7 : -55.0803950011521
column 8 : 0.0
column 9 : 0.0
column 10 : 0.0
column 11 : 59024.068622439896
column 12 : 0.0
column 13 : -240.15554145890167
column 14 : -12745.632994028303
column 15 : 0.0
column 16 : -1217.2552850604727
column 17 : 0.0
column 18 : 0.0
column 19 : 10400.571767577618
column 20 : 0.0
column 21 : 0.0
column 22 : 0.0
column 23 : -116.70677833728621
column 24 : 0.0
column 25 : 562791.5325195512
column 26 : 0.0
column 27 : 26637.78354870597
column 28 : 4254.399688846217
column 29 : -162.26327330175513
column 30 : 0.0
column 31 : -2399.714765633049
column 32 : 0.0
column 33 : 0.0
column 34 : 8874.698423978487
column 35 : 0.0
column 36 : 0.0
column 37 : 0.0
column 38 : 0.0
column 39 : -9420.075082235075
column 40 : 0.0
column 41 : -105.68729116468963
column 42 : 0.0
column 43 : -51.449059947375474
column 44 : 0.0
co

In [ ]:
pure_object_cols_test = []

for i in range(len(object_columns)):
    numeric_values = pd.to_numeric(test_prop[object_columns[i]], errors='coerce')
    column_sum = numeric_values.sum()
    print('column',i,':', column_sum)
    if column_sum == 0.0:
        pure_object_cols_test.append(object_columns[i])


column 0 : 0.0
column 1 : 0.0
column 2 : -100.19345869424967
column 3 : 0.0
column 4 : 74.32562316528222
column 5 : 0.0
column 6 : -30.73792218754016
column 7 : -11.233789300205425
column 8 : 0.0
column 9 : 0.0
column 10 : 0.0
column 11 : 7941.828816098412
column 12 : 0.0
column 13 : -14.587680739152969
column 14 : -2218.546703887571
column 15 : 0.0
column 16 : -181.55377746553165
column 17 : 0.0
column 18 : 0.0
column 19 : 1412.2234403035084
column 20 : 0.0
column 21 : 0.0
column 22 : 0.0
column 23 : -21.07703104157801
column 24 : 0.0
column 25 : 79058.46185636106
column 26 : 0.0
column 27 : 3721.253484139411
column 28 : 583.1885164705277
column 29 : -17.072277852100637
column 30 : 0.0
column 31 : -290.24847275597017
column 32 : 0.0
column 33 : 0.0
column 34 : 1227.4177306317197
column 35 : 0.0
column 36 : 0.0
column 37 : 0.0
column 38 : 0.0
column 39 : -1369.5794671560068
column 40 : 0.0
column 41 : -13.73309099996576
column 42 : 0.0
column 43 : -2.7496436917190605
column 44 : 0.0
co

In [ ]:
pure_object_cols = pure_object_cols_train + pure_object_cols_test

pure_object_cols = set(pure_object_cols)
pure_object_cols = list(pure_object_cols)


len(pure_object_cols_train), len(pure_object_cols_test), len(pure_object_cols)

(313, 317, 319)

In [ ]:
len(object_columns), len(pure_object_cols)

# 575개 중에 pure object cols 313개는 object(catboost로) / 나머지는 머신러닝으로 -> 앙상블?

(575, 319)

In [ ]:
train[pure_object_cols]


,Mor12p,Mor06p,MINsssGeH,Mor21m,SpAD_Dt,FNSA1,SpMax_Dt,Mor29m,Mor02m,Mor18p,...,MINsssdAs,Mor20se,MAXsssAs,MAXssNH2,Mor23se,Mor19m,Mor08se,Mor22p,Mor10v,Mor16m
0,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssGeH),missing 3D coordinate (GeomDiameter/mordred._g...,module networkx has no attribute biconnected_c...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),module networkx has no attribute biconnected_c...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,...,min() arg is an empty sequence (MINsssdAs),missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssAs),max() arg is an empty sequence (MAXssNH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...
1,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssGeH),missing 3D coordinate (GeomDiameter/mordred._g...,module networkx has no attribute biconnected_c...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),module networkx has no attribute biconnected_c...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,...,min() arg is an empty sequence (MINsssdAs),missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssAs),max() arg is an empty sequence (MAXssNH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...
2,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssGeH),missing 3D coordinate (GeomDiameter/mordred._g...,module networkx has no attribute biconnected_c...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),module networkx has no attribute biconnected_c...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,...,min() arg is an empty sequence (MINsssdAs),missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssAs),max() arg is an empty sequence (MAXssNH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...
3,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssGeH),missing 3D coordinate (GeomDiameter/mordred._g...,module networkx has no attribute biconnected_c...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),module networkx has no attribute biconnected_c...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,...,min() arg is an empty sequence (MINsssdAs),missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssAs),max() arg is an empty sequence (MAXssNH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred.

In [ ]:
train[pure_object_cols]

,Mor12p,Mor06p,MINsssGeH,Mor21m,SpAD_Dt,FNSA1,SpMax_Dt,Mor29m,Mor02m,Mor18p,...,MINsssdAs,Mor20se,MAXsssAs,MAXssNH2,Mor23se,Mor19m,Mor08se,Mor22p,Mor10v,Mor16m
0,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssGeH),missing 3D coordinate (GeomDiameter/mordred._g...,module networkx has no attribute biconnected_c...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),module networkx has no attribute biconnected_c...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,...,min() arg is an empty sequence (MINsssdAs),missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssAs),max() arg is an empty sequence (MAXssNH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...
1,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssGeH),missing 3D coordinate (GeomDiameter/mordred._g...,module networkx has no attribute biconnected_c...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),module networkx has no attribute biconnected_c...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,...,min() arg is an empty sequence (MINsssdAs),missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssAs),max() arg is an empty sequence (MAXssNH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...
2,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssGeH),missing 3D coordinate (GeomDiameter/mordred._g...,module networkx has no attribute biconnected_c...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),module networkx has no attribute biconnected_c...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,...,min() arg is an empty sequence (MINsssdAs),missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssAs),max() arg is an empty sequence (MAXssNH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...
3,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,min() arg is an empty sequence (MINsssGeH),missing 3D coordinate (GeomDiameter/mordred._g...,module networkx has no attribute biconnected_c...,missing 3D coordinate (PNSA1/AtomicSurfaceArea),module networkx has no attribute biconnected_c...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,...,min() arg is an empty sequence (MINsssdAs),missing 3D coordinate (GeomDiameter/mordred._g...,max() arg is an empty sequence (MAXsssAs),max() arg is an empty sequence (MAXssNH2),missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred._g...,missing 3D coordinate (GeomDiameter/mordred.

### 2) float object columns -> Catboost (256개)

In [ ]:
float_object_cols = [x for x in object_columns if x not in pure_object_cols]
float_object_cols

['AATSC8dv',
 'MDEN-33',
 'MATS7se',
 'MATS6i',
 'AATS7Z',
 'MAXdssC',
 'AATSC7m',
 'AATSC6dv',
 'AATS8dv',
 'MATS6p',
 'AATS8i',
 'AATS7se',
 'AATS8p',
 'MATS8pe',
 'MAXddssS',
 'MINaaN',
 'AATSC7v',
 'MATS6s',
 'ATSC8c',
 'MINdCH2',
 'MAXtN',
 'GATS6i',
 'GATS6se',
 'MINsNH2',
 'MATS1c',
 'AATSC8Z',
 'MINddssS',
 'AATSC8s',
 'GATS7v',
 'AATS6d',
 'AATSC8m',
 'AATSC7d',
 'MAXsF',
 'MINdS',
 'MATS6m',
 'MAXdCH2',
 'MATS6c',
 'MDEO-12',
 'MDEC-22',
 'MAXsssCH',
 'MAXaasC',
 'MAXddsN',
 'MATS7are',
 'GATS8i',
 'AATS8m',
 'GATS7p',
 'MATS6se',
 'MAXaaS',
 'MATS7p',
 'MDEO-22',
 'MINsF',
 'MINtsC',
 'MATS7d',
 'MATS7dv',
 'GATS1c',
 'GATS7c',
 'GATS7are',
 'AATSC1c',
 'GATS6pe',
 'AATSC7are',
 'GATS7m',
 'AATS6are',
 'RNCG',
 'MATS7i',
 'MAXdNH',
 'MINdsCH',
 'MATS6pe',
 'MATS6v',
 'GATS6d',
 'GATS6dv',
 'MDEC-44',
 'AATS8d',
 'MAXdssS',
 'MINsBr',
 'GATS8are',
 'GATS8c',
 'AATS6p',
 'MDEC-23',
 'AATS6v',
 'MAXtsC',
 'MAXdsN',
 'GATS5c',
 'GATS8m',
 'Vabc',
 'MDEC-33',
 'MDEN-11',
 'AATSC7

In [ ]:
len(float_object_cols)

256

In [ ]:
train[float_object_cols]

,AATSC8dv,MDEN-33,MATS7se,MATS6i,AATS7Z,MAXdssC,AATSC7m,AATSC6dv,AATS8dv,MATS6p,...,AATS7i,MINsOH,GATS8d,AATS6dv,GATS8v,AATS6Z,MINsCH3,AATS8se,AATSC8se,MDEC-13
0,0.6483399079552924,float division by zero (MDEN-33),0.155872,-0.039489,15.018692,-0.21806951793062868,1.380469,-0.45428,2.900641025641026,-0.005236,...,162.963227,min() arg is an empty sequence (MINsOH),0.7200190355329951,2.717333,0.8361791370677145,14.512,1.9229353985270639,7.5651532884615404,0.007238681767751504,5.8076276350714044
1,-0.45925925925925903,float division by zero (MDEN-33),0.077679,0.08152,15.964706,0.0853306353405563,-6.749905,0.0607,1.4,0.153694,...,157.558607,min() arg is an empty sequence (MINsOH),0.9938916256157634,2.185185,1.1628776838400774,15.324074,1.9712226001511715,7.384859573333333,0.001871056958333344,5.61419311594246
2,-1.1276356666005687,0.7937005259840998,-0.155452,-0.080973,11.197674,max() arg is an empty sequence (MAXdssC),-2.435919,-0.528667,2.3703703703703702,-0.07473,...,172.290275,min() arg is an empty sequence (MINsOH),1.1979783621574667,2.262136,0.9679830977406495,12.533981,2.2292460165629358,7.613644740740741,-0.005302947187520656,0.47473620878083905
3,-0.4759524167747899,0.6850067105944414,0.063621,-0.07761,14.195402,max() arg is an empty sequence (MAXdssC),2.717983,0.330945,3.00709219858156,-0.041391,...,168.254616,min() arg is an empty sequence (MINsOH),1.1892291594000692,2.677419,0.7554952713218305,14.424731,1.7527397714518926,7.690458212765958,-0.01372508153123568,4.989429706723068
4,0.2987549696589244,float division by zero (MDEN-33),-0.077712,-0.168633,11.259259,-0.019074074074073577,-1.644396,-0.889149,1.5932203389830508,-0.181995,...,162.052654,min() arg is an empty sequence (MINsOH),0.48536209553158716,2.662651,1.2482287378164674,14.180723,1.9678829680020156,7.3251919999999995,0.006447660179953947,3.452337341766291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,-1.216709672374697,float division by zero (MDEN-33),-0.136013,0.008383,19.444444,max() arg is an empty sequence (MAXdssC),-0.272942,0.48288,4.810344827586207,0.163342,...,162.893261,min() arg is an empty sequence (MINsOH),1.0412738774497643,4.65,0.8912558298994588,16.1375,2.0889870182406467,7.774875999999997,-0.018639348659003848,3.7623500633231233
3450,0.176852,0.7937005259840998,0.01393,0.029704,16.30303,-0.1728884269757287,-4.221044,-0.369649,2.889984,-0.052736,...,158.037809,min() arg is an empty sequence (MINsOH),0.896656,3.508021,0.856052,18.491979,1.8070926141815091,7.495607,0.005683,3.4398675102497274
3451,0.052227810801199044,float division by zero (MDEN-33),-0.08138,0.000292,17.900621,max() arg is an empty sequence (MAXdssC),-3.655255,-0.066641,3.6831275720164616,-0.049382,...,156.715134,min() arg is an empty sequence (MINsOH),0.8000464468183933,5.021645,1.003689482042737,19.376623,1.0529719014684744,7.601226111111109,-0.0031159927983538407,7.48986394600527
3452,-0.04938271604938271,0.25,-0.177223,-0.088507,14.216216,0.06524894312637852,-0.738231,-0.020833,4.54320987654321,-0.04761,...,170.444985,min() arg is an empty sequence (MINsOH),1.0521885521885523,2.909722,1.074697323785847,13.138889,min() arg is an empty sequence (MINsCH3),7.935928888888889,0.010179502926224927,float division by zero (MDEC-13)


In [ ]:
row_types = []

#iterrows
for index, row in train_prop.iterrows():
    row_data_types = row.apply(lambda x: type(x))
    row_types.append(row_data_types)

row_types_df = pd.DataFrame(row_types, columns=train_prop.columns)

row_types_df

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,ABC,ABCGG,nAcid,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,...,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>
1,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,...,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>
2,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,...,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>
3,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,...,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>
4,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,...,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,...,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>
3450,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,...,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>
3451,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,...,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>
3452,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,...,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>


In [ ]:
row_types_df[pure_object_cols]

,Mor12p,Mor06p,MINsssGeH,Mor21m,SpAD_Dt,FNSA1,SpMax_Dt,Mor29m,Mor02m,Mor18p,...,MINsssdAs,Mor20se,MAXsssAs,MAXssNH2,Mor23se,Mor19m,Mor08se,Mor22p,Mor10v,Mor16m
0,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3450,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3451,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3452,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [ ]:
row_types_df[float_object_cols]

,AATSC8dv,MDEN-33,MATS7se,MATS6i,AATS7Z,MAXdssC,AATSC7m,AATSC6dv,AATS8dv,MATS6p,...,AATS7i,MINsOH,GATS8d,AATS6dv,GATS8v,AATS6Z,MINsCH3,AATS8se,AATSC8se,MDEC-13
0,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3450,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,...,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>
3451,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3452,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'str'>,<class 'float'>,...,<class 'float'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [ ]:
#예시
# pd.DataFrame(train_prop['AATS6dv'])

- float division by zero 다수 포함. 이 부분을 그대로 cat으로 학습시키는게 좋겠다고 판단함

In [ ]:
# train_prop[float_object_cols].astype(float)
# float division by zero

# train_prop[float_object_cols] = train_prop[float_object_cols].apply(lambda x: 0 if isinstance(x, str) else x)

train_prop[float_object_cols]

,AATSC8dv,MDEN-33,MATS7se,MATS6i,AATS7Z,MAXdssC,AATSC7m,AATSC6dv,AATS8dv,MATS6p,...,AATS7i,MINsOH,GATS8d,AATS6dv,GATS8v,AATS6Z,MINsCH3,AATS8se,AATSC8se,MDEC-13
0,0.6483399079552924,float division by zero (MDEN-33),0.155872,-0.039489,15.018692,-0.21806951793062868,1.380469,-0.45428,2.900641025641026,-0.005236,...,162.963227,min() arg is an empty sequence (MINsOH),0.7200190355329951,2.717333,0.8361791370677145,14.512,1.9229353985270639,7.5651532884615404,0.007238681767751504,5.8076276350714044
1,-0.45925925925925903,float division by zero (MDEN-33),0.077679,0.08152,15.964706,0.0853306353405563,-6.749905,0.0607,1.4,0.153694,...,157.558607,min() arg is an empty sequence (MINsOH),0.9938916256157634,2.185185,1.1628776838400774,15.324074,1.9712226001511715,7.384859573333333,0.001871056958333344,5.61419311594246
2,-1.1276356666005687,0.7937005259840998,-0.155452,-0.080973,11.197674,max() arg is an empty sequence (MAXdssC),-2.435919,-0.528667,2.3703703703703702,-0.07473,...,172.290275,min() arg is an empty sequence (MINsOH),1.1979783621574667,2.262136,0.9679830977406495,12.533981,2.2292460165629358,7.613644740740741,-0.005302947187520656,0.47473620878083905
3,-0.4759524167747899,0.6850067105944414,0.063621,-0.07761,14.195402,max() arg is an empty sequence (MAXdssC),2.717983,0.330945,3.00709219858156,-0.041391,...,168.254616,min() arg is an empty sequence (MINsOH),1.1892291594000692,2.677419,0.7554952713218305,14.424731,1.7527397714518926,7.690458212765958,-0.01372508153123568,4.989429706723068
4,0.2987549696589244,float division by zero (MDEN-33),-0.077712,-0.168633,11.259259,-0.019074074074073577,-1.644396,-0.889149,1.5932203389830508,-0.181995,...,162.052654,min() arg is an empty sequence (MINsOH),0.48536209553158716,2.662651,1.2482287378164674,14.180723,1.9678829680020156,7.3251919999999995,0.006447660179953947,3.452337341766291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,-1.216709672374697,float division by zero (MDEN-33),-0.136013,0.008383,19.444444,max() arg is an empty sequence (MAXdssC),-0.272942,0.48288,4.810344827586207,0.163342,...,162.893261,min() arg is an empty sequence (MINsOH),1.0412738774497643,4.65,0.8912558298994588,16.1375,2.0889870182406467,7.774875999999997,-0.018639348659003848,3.7623500633231233
3450,0.176852,0.7937005259840998,0.01393,0.029704,16.30303,-0.1728884269757287,-4.221044,-0.369649,2.889984,-0.052736,...,158.037809,min() arg is an empty sequence (MINsOH),0.896656,3.508021,0.856052,18.491979,1.8070926141815091,7.495607,0.005683,3.4398675102497274
3451,0.052227810801199044,float division by zero (MDEN-33),-0.08138,0.000292,17.900621,max() arg is an empty sequence (MAXdssC),-3.655255,-0.066641,3.6831275720164616,-0.049382,...,156.715134,min() arg is an empty sequence (MINsOH),0.8000464468183933,5.021645,1.003689482042737,19.376623,1.0529719014684744,7.601226111111109,-0.0031159927983538407,7.48986394600527
3452,-0.04938271604938271,0.25,-0.177223,-0.088507,14.216216,0.06524894312637852,-0.738231,-0.020833,4.54320987654321,-0.04761,...,170.444985,min() arg is an empty sequence (MINsOH),1.0521885521885523,2.909722,1.074697323785847,13.138889,min() arg is an empty sequence (MINsCH3),7.935928888888889,0.010179502926224927,float division by zero (MDEC-13)


### 3) SMILES, ID -> 보류

In [ ]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.0100,50.6800,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.2700,50.5900,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.5860,80.8920,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.7100,2.0000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.2700,99.9900,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,TRAIN_1833,Cc1ccc(-c2cc(-c3nc4c5ccccc5ncn4n3)[nH]n2)c(C)c1,28.4630,24.1955,4.609,340.381,4,1,2,4.736,...,10.344674,77.430819,340.143645,8.098658,1730,43,148.0,180.0,6.611111,5.500000
3450,TRAIN_0898,Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4...,56.0435,66.1260,4.282,553.480,5,0,7,4.009,...,10.593756,88.164476,552.169496,8.120140,4874,63,204.0,242.0,12.055556,8.222222
3451,TRAIN_2575,Cc1ccccc1-c1nc2ccccc2cc1C(C)n1c(=O)[nH]c2c(S(C...,56.3130,63.7320,4.304,459.520,6,1,4,4.304,...,10.677177,84.226168,459.136511,8.502528,2888,61,186.0,227.0,10.895833,6.944444
3452,TRAIN_0216,N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1,70.2150,93.8400,-1.133,385.417,7,0,4,-1.133,...,10.177818,79.094663,385.175004,7.552451,2262,42,148.0,175.0,8.000000,6.222222


## Float Columns (총 1258개)

In [ ]:
float_columns = [x for x in train_prop.columns if x not in object_columns]

In [ ]:
float_columns

['AlogP',
 'Molecular_Weight',
 'Num_H_Acceptors',
 'Num_H_Donors',
 'Num_RotatableBonds',
 'LogD',
 'Molecular_PolarSurfaceArea',
 'ABC',
 'ABCGG',
 'nAcid',
 'nBase',
 'SpAbs_A',
 'SpMax_A',
 'SpDiam_A',
 'SpAD_A',
 'SpMAD_A',
 'LogEE_A',
 'VE1_A',
 'VE2_A',
 'VE3_A',
 'VR1_A',
 'VR2_A',
 'VR3_A',
 'nAromAtom',
 'nAromBond',
 'nAtom',
 'nHeavyAtom',
 'nSpiro',
 'nBridgehead',
 'nHetero',
 'nH',
 'nB',
 'nC',
 'nN',
 'nO',
 'nS',
 'nP',
 'nF',
 'nCl',
 'nBr',
 'nI',
 'nX',
 'ATS0dv',
 'ATS1dv',
 'ATS2dv',
 'ATS3dv',
 'ATS4dv',
 'ATS5dv',
 'ATS6dv',
 'ATS7dv',
 'ATS8dv',
 'ATS0d',
 'ATS1d',
 'ATS2d',
 'ATS3d',
 'ATS4d',
 'ATS5d',
 'ATS6d',
 'ATS7d',
 'ATS8d',
 'ATS0s',
 'ATS1s',
 'ATS2s',
 'ATS3s',
 'ATS4s',
 'ATS5s',
 'ATS6s',
 'ATS7s',
 'ATS8s',
 'ATS0Z',
 'ATS1Z',
 'ATS2Z',
 'ATS3Z',
 'ATS4Z',
 'ATS5Z',
 'ATS6Z',
 'ATS7Z',
 'ATS8Z',
 'ATS0m',
 'ATS1m',
 'ATS2m',
 'ATS3m',
 'ATS4m',
 'ATS5m',
 'ATS6m',
 'ATS7m',
 'ATS8m',
 'ATS0v',
 'ATS1v',
 'ATS2v',
 'ATS3v',
 'ATS4v',
 'ATS5v',
 '

In [ ]:
len(float_columns)

1258

# 학습, 검증, 추론

## 데이터 분리 ***

- 1) pure object columns 삭제

In [ ]:
train_prop.drop(columns = pure_object_cols, inplace=True)
test_prop.drop(columns = pure_object_cols, inplace=True)


In [ ]:
train_prop.shape, test_prop.shape

((3454, 1514), (483, 1514))

In [ ]:
# cols = ['mol','mode_label_proba']

# train = train.drop(columns = cols,axis=1)
# test = test.drop(columns = cols,axis=1)

- 2) float object columns

In [ ]:
train_prop_object = train_prop[float_object_cols]
test_prop_object = test_prop[float_object_cols]


- 3) SMILES, ID

In [ ]:
train.head()

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111


In [ ]:
train_prop_smiles = train[['id','SMILES']]
test_prop_smiles = test[['id','SMILES']]

- 4) float columns

In [ ]:
train_prop_float = train_prop[float_columns]
test_prop_float = test_prop[float_columns]

In [ ]:
from sklearn.metrics import mean_squared_error

def rmse(y_valid, pred):
    mse = mean_squared_error(y_valid, pred)
    return np.sqrt(mse)

In [ ]:
from sklearn.metrics import make_scorer

rmse_score = make_scorer(rmse, greater_is_better=False)

### A. object(float) columns

In [ ]:
train_prop_object.isna().sum().sum(), test_prop_object.isna().sum().sum()

(0, 0)

#### CatBoost

In [ ]:
!pip install CatBoost

In [ ]:
from catboost import CatBoostClassifier


In [ ]:
import pandas as pd

# train_prop_object의 모든 열을 처리
for column_name in train_prop_object.columns:
    train_prop_object[column_name] = pd.to_numeric(train_prop_object[column_name], errors='coerce')

# test_prop_object의 모든 열을 처리
for column_name in test_prop_object.columns:
    test_prop_object[column_name] = pd.to_numeric(test_prop_object[column_name], errors='coerce')



In [ ]:
train_prop_object.isna().sum().sum(), test_prop_object.isna().sum().sum()

(196428, 27480)

In [ ]:
train_prop_object.shape[0] * train_prop_object.shape[1]



880770

In [ ]:
train_prop_object.fillna(0, inplace=True)
test_prop_object.fillna(0, inplace=True)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop_object, target_1)):
    # 학습데이터
    x_train = train_prop_object.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop_object.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    model = CatBoostRegressor(random_state=SEED, verbose=0)
    model.fit(x_train, y_train_1)
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    #HLM
    model = CatBoostRegressor(random_state=SEED, verbose=0)
    model.fit(x_train, y_train_2)
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

5it [12:03, 144.76s/it]

[31.845847257436787, 32.568143462806326, 33.018936493098664, 31.322089283807486, 32.120797577145446]


32.17516281485894

In [ ]:
model.fit(train_prop_object, target_1)
cat_pred_1 = model.predict(test_prop_object)

model.fit(train_prop_object, target_2)
cat_pred_2 = model.predict(test_prop_object)

### B. float columns

In [ ]:
train_prop_float.isna().sum().sum(), test_prop_float.isna().sum().sum()

(2, 1)

In [ ]:
# 결측치 채우기
train_prop_float["AlogP"] = np.where(pd.isna(train_prop_float["AlogP"]), train_prop_float["LogD"], train_prop_float["AlogP"])
test_prop_float["AlogP"] = np.where(pd.isna(test_prop_float["AlogP"]), test_prop_float["LogD"], test_prop_float["AlogP"])

- 스케일링

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max 스케일링을 위한 객체 생성
scaler = MinMaxScaler()

# Train 데이터를 스케일링하고 변환
train_prop_float = scaler.fit_transform(train_prop_float)

# Test 데이터를 스케일링하고 변환
test_prop_float = scaler.transform(test_prop_float)


In [ ]:
train_prop_float = pd.DataFrame(train_prop_float)
train_prop_float

,0,1,2,3,4,5,6,7,8,9,...,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257
0,0.488883,0.408759,0.416667,0.250,0.470588,0.525975,0.546835,0.405597,0.417590,0.000000,...,0.544667,0.579584,0.408596,0.137182,0.214867,0.305263,0.373016,0.345794,0.345075,0.466563
1,0.394950,0.231359,0.166667,0.125,0.117647,0.439098,0.336496,0.264882,0.279238,0.000000,...,0.485644,0.436791,0.231203,0.127029,0.072064,0.252632,0.253968,0.242991,0.222861,0.261275
2,0.345312,0.224110,0.416667,0.000,0.176471,0.392184,0.283695,0.292099,0.263407,0.666667,...,0.566000,0.451774,0.224119,0.110023,0.087006,0.263158,0.285714,0.283489,0.126528,0.301711
3,0.619183,0.577331,0.500000,0.000,0.294118,0.543238,0.428154,0.593912,0.522812,0.000000,...,0.725217,0.690095,0.577135,0.104858,0.384150,0.526316,0.571429,0.551402,0.411574,0.594090
4,0.409255,0.172104,0.250000,0.000,0.058824,0.452286,0.187773,0.241146,0.229077,0.000000,...,0.509591,0.209825,0.172085,0.122153,0.061850,0.221053,0.230159,0.221184,0.178289,0.242613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,0.605222,0.301135,0.333333,0.125,0.117647,0.644022,0.328255,0.400066,0.353467,0.000000,...,0.633958,0.559821,0.301096,0.161673,0.153395,0.336842,0.396825,0.392523,0.195543,0.370140
3450,0.577042,0.682652,0.416667,0.000,0.411765,0.585918,0.300608,0.657112,0.625926,0.000000,...,0.718524,0.719286,0.680891,0.162977,0.450728,0.547368,0.619048,0.585670,0.477354,0.674961
3451,0.578938,0.514433,0.500000,0.125,0.235294,0.609495,0.528437,0.554507,0.550021,0.000000,...,0.746847,0.660777,0.514244,0.186196,0.262909,0.526316,0.547619,0.538941,0.417326,0.531882
3452,0.110393,0.381764,0.583333,0.000,0.235294,0.174952,0.505055,0.421798,0.415727,0.000000,...,0.577309,0.584540,0.381759,0.128507,0.203707,0.326316,0.396825,0.376947,0.267434,0.451011


In [ ]:
test_prop_float = pd.DataFrame(test_prop_float)
test_prop_float

,0,1,2,3,4,5,6,7,8,9,...,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257
0,0.435626,0.338954,0.333333,0.250,0.411765,0.476103,0.428921,0.343099,0.335359,0.0,...,0.495111,0.485416,0.338802,0.091685,0.141763,0.242105,0.309524,0.280374,0.242991,0.385692
1,0.258445,0.354877,0.416667,0.000,0.176471,0.312260,0.311772,0.401242,0.394503,0.0,...,0.629119,0.540223,0.354851,0.128609,0.165027,0.357895,0.388889,0.376947,0.259166,0.423017
2,0.576525,0.313726,0.333333,0.500,0.294118,0.608376,0.429400,0.370016,0.337512,0.0,...,0.519708,0.309983,0.313691,0.118442,0.169945,0.273684,0.333333,0.299065,0.287563,0.395023
3,0.362720,0.310045,0.416667,0.000,0.117647,0.408967,0.373581,0.396886,0.361725,0.0,...,0.614867,0.560326,0.310035,0.181057,0.148005,0.326316,0.388889,0.383178,0.163911,0.379471
4,0.313082,0.324475,0.333333,0.000,0.117647,0.279012,0.277466,0.383519,0.361273,0.0,...,0.646057,0.536702,0.324457,0.107583,0.146397,0.378947,0.380952,0.380062,0.304817,0.385692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,0.570579,0.240375,0.166667,0.125,0.411765,0.601742,0.248622,0.257693,0.306877,0.0,...,0.470257,0.444321,0.240349,0.027504,0.097409,0.221053,0.238095,0.221184,0.254853,0.339036
479,0.155636,0.292214,0.416667,0.000,0.058824,0.126758,0.320636,0.329945,0.351373,0.0,...,0.615006,0.503620,0.292213,0.085277,0.110838,0.326316,0.325397,0.320872,0.329619,0.334370
480,0.362461,0.317251,0.250000,0.125,0.176471,0.408728,0.318528,0.384347,0.390170,0.0,...,0.668779,0.542689,0.317216,0.141032,0.128712,0.389474,0.380952,0.383178,0.304817,0.382582
481,0.276112,0.302479,0.250000,0.250,0.117647,0.299313,0.318145,0.230130,0.246490,0.0,...,0.474406,0.205264,0.300814,0.358055,0.067902,0.210526,0.214286,0.199377,0.287563,0.242613


In [ ]:
# import joblib

# # 모델 저장
# joblib.dump(mlm_model, 'mlm_model.pkl')
# joblib.dump(hlm_model, 'hlm_model.pkl')

#### AdaBoost

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold

# cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

# scores = []
# for tri, val in tqdm(cv.split(train_prop_float, target_1)):
#     # 학습데이터
#     x_train = train_prop_float.iloc[tri]
#     y_train_1 = target_1.iloc[tri]
#     y_train_2 = target_2.iloc[tri]

#     # 검증데이터
#     x_valid = train_prop_float.iloc[val]
#     y_valid_1 = target_1.iloc[val]
#     y_valid_2 = target_2.iloc[val]

#     # MLM
#     model = AdaBoostRegressor(random_state=SEED)
#     model.fit(x_train, y_train_1)
#     pred = model.predict(x_valid)
#     score_1 = rmse(y_valid_1, pred)

#     #HLM
#     model = AdaBoostRegressor(random_state=SEED)
#     model.fit(x_train, y_train_2)
#     pred = model.predict(x_valid)
#     score_2 = rmse(y_valid_2, pred)

#     score = 0.5*score_1 + 0.5*score_2
#     scores.append(score)

# print(scores)
# np.mean(scores)

In [ ]:
# model.fit(train_prop_float, target_1)
# ada_pred_1 = model.predict(test_prop_float)

# model.fit(train_prop_float, target_2)
# ada_pred_2 = model.predict(test_prop_float)

#### randomforest

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []
for tri, val in tqdm(cv.split(train_prop_float, target_1)):
    # 학습데이터
    x_train = train_prop_float.iloc[tri]
    y_train_1 = target_1.iloc[tri]
    y_train_2 = target_2.iloc[tri]

    # 검증데이터
    x_valid = train_prop_float.iloc[val]
    y_valid_1 = target_1.iloc[val]
    y_valid_2 = target_2.iloc[val]

    # MLM
    model = RandomForestRegressor(random_state=SEED)
    model.fit(x_train, y_train_1)
    pred = model.predict(x_valid)
    score_1 = rmse(y_valid_1, pred)

    #HLM
    model = RandomForestRegressor(random_state=SEED)
    model.fit(x_train, y_train_2)
    pred = model.predict(x_valid)
    score_2 = rmse(y_valid_2, pred)

    score = 0.5*score_1 + 0.5*score_2
    scores.append(score)

print(scores)
np.mean(scores)

5it [46:39, 559.97s/it]

[30.978366949192015, 32.575945789114286, 31.787306579152258, 30.688974175121523, 31.32539794849395]


31.47119828821481

In [ ]:
model.fit(train_prop_float, target_1)
rf_pred_1 = model.predict(test_prop_float)

model.fit(train_prop_float, target_2)
rf_pred_2 = model.predict(test_prop_float)

#### XGBRegressor

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold

# cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

# scores = []
# for tri, val in tqdm(cv.split(train_prop, target_1)):
#     # 학습데이터
#     x_train = train_prop.iloc[tri]
#     y_train_1 = target_1.iloc[tri]
#     y_train_2 = target_2.iloc[tri]

#     # 검증데이터
#     x_valid = train_prop.iloc[val]
#     y_valid_1 = target_1.iloc[val]
#     y_valid_2 = target_2.iloc[val]

#     # MLM
#     model = XGBRegressor(random_state=SEED)
#     model.fit(x_train, y_train_1, eval_set=[(x_valid, y_valid_1)], eval_metric='rmse')
#     pred = model.predict(x_valid)
#     score_1 = rmse(y_valid_1, pred)

#     # HLM
#     model = XGBRegressor(random_state=SEED)
#     model.fit(x_train, y_train_2, eval_set=[(x_valid, y_valid_2)], eval_metric='rmse')
#     pred = model.predict(x_valid)
#     score_2 = rmse(y_valid_2, pred)

#     score = 0.5*score_1 + 0.5*score_2
#     scores.append(score)

# print(scores)
# np.mean(scores)

In [ ]:
# model.fit(train_prop, target_1)
# xgb_pred_1 = model.predict(test_prop)

# model.fit(train_prop, target_2)
# xgb_pred_2 = model.predict(test_prop)

#### LGBMRegressor

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold

# cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

# scores = []
# for tri, val in tqdm(cv.split(train_prop, target_1)):
#     # 학습데이터
#     x_train = train_prop.iloc[tri]
#     y_train_1 = target_1.iloc[tri]
#     y_train_2 = target_2.iloc[tri]

#     # 검증데이터
#     x_valid = train_prop.iloc[val]
#     y_valid_1 = target_1.iloc[val]
#     y_valid_2 = target_2.iloc[val]

#     # MLM
#     model = LGBMRegressor(random_state=SEED, objective='regression')
#     model.fit(x_train, y_train_1, eval_set=[(x_valid, y_valid_1)], eval_metric="rmse")
#     pred = model.predict(x_valid)
#     score_1 = rmse(y_valid_1, pred)

#     # HLM
#     model = LGBMRegressor(random_state=SEED, objective='regression')
#     model.fit(x_train, y_train_2, eval_set=[(x_valid, y_valid_2)], eval_metric="rmse")
#     pred = model.predict(x_valid)
#     score_2 = rmse(y_valid_2, pred)


#     score = 0.5*score_1 + 0.5*score_2
#     scores.append(score)

# print(scores)
# np.mean(scores)

In [ ]:
# model.fit(train_prop, target_1)
# lgbm_pred_1 = model.predict(test_prop)

# model.fit(train_prop, target_2)
# lgbm_pred_2 = model.predict(test_prop)

In [ ]:
# mlm_pred = (cat_pred_1 + ada_pred_1 + rf_pred_1 + xgb_pred_1 + lgbm_pred_1) / 5
# hlm_pred = (cat_pred_2 + ada_pred_2 +rf_pred_2 + xgb_pred_2 + lgbm_pred_2) / 5

In [ ]:
mlm_pred = (cat_pred_1)*0.5 + (rf_pred_1)*0.5
hlm_pred = (cat_pred_2)*0.5 + (rf_pred_2)* 0.5

In [ ]:
submission['MLM'] = mlm_pred
submission['HLM'] = hlm_pred

In [ ]:
submission

,id,MLM,HLM
0,TEST_000,29.346688,47.805495
1,TEST_001,49.700280,74.044434
2,TEST_002,39.409920,54.119813
3,TEST_003,45.239855,67.436936
4,TEST_004,50.288453,70.644674
...,...,...,...
478,TEST_478,26.633371,47.933798
479,TEST_479,61.370739,75.454900
480,TEST_480,44.359946,64.263805
481,TEST_481,53.342733,68.428288


In [ ]:
submission.to_csv("submission_ML20(cat).csv", index=False)

In [ ]:
submission

In [ ]:
sub_16 = pd.read_csv('submission_ML16(preprocess).csv')

In [ ]:
sub_16

In [ ]:
sub_10 = pd.read_csv('submission_ML10(preprocess).csv')

In [ ]:
sub_10